In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
from os.path import join, isdir

import pandas as pd
import geopandas as gpd
import numpy as np

from covidcaremap.data import (processed_data_path, 
                               external_data_path,
                               published_data_path,
                               local_data_path)

from rapidfuzz import fuzz, process

In [3]:
ccm_matching_df = pd.read_csv(external_data_path('covidcaremap_facility_matching_20200407.csv'))

In [4]:
ccm_matching_df.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID,DH name lookup,DH address lookup,HCRIS name lookup,HCRIS address lookup
0,IU HEALTH UNIVERSITY HOSPITAL,GENERAL ACUTE CARE,550 UNIVERSITY BLVD,NaN,INDIANAPOLIS,IN,46202,MARION,39.775283,-86.176562,100,NaN,NaN,100,NaN,NaN,NaN,NaN
1,HEALTHSOUTH NEW ENGLAND REHABILIATION AT BEVERLY,REHABILITATION,"800 CUMMINGS CENTER, 1ST FL, UNIT147",NaN,BEVERLY,MA,1915,ESSEX,42.560987,-70.887236,10001915,NaN,NaN,10001915,NaN,NaN,NaN,NaN
2,ST. JOSEPH'S HOSPITAL,CRITICAL ACCESS,12886 TROXLER AVENUE,NaN,HIGHLAND,IL,62249,MADISON,38.754584,-89.667598,100062249,NaN,NaN,100062249,NaN,NaN,NaN,NaN
3,HOSP SAN CARLOS BORROMEO,GENERAL ACUTE CARE,550 CONCEPCION VERA AYALA,NaN,MOCA,PR,676,MOCA,18.390194,-67.109969,1000676,4294.0,400111.0,1000676,Hospital San Carlos Borromeo,"Calle Concepcion Vera Ayala #550 S, Moca, PR",HOSPITAL SAN CARLOS BORROMEO,"68, MOCA, PR"
4,RIVERVIEW MEDICAL CENTER,GENERAL ACUTE CARE,1 RIVERVIEW PLAZA,NaN,RED BANK,NJ,7701,MONMOUTH,40.353982,-74.063843,10007701,2512.0,310034.0,10007701,Riverview Medical Center,"1 Riverview Plz, Red Bank, NJ",RIVERVIEW MEDICAL CENTER,"RIVERVIEW PLAZA, RED BANK, NJ"


In [5]:
ccm_csv_df = pd.read_csv(
    published_data_path('us_healthcare_capacity-facility-CovidCareMap.csv'), 
    dtype={'HCRIS-Provider Number': object, 
           'DH-OBJECTID': object,
           'CCM_ID': object
          }
)

In [6]:
ccm_csv_df.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,...,ICU Bed Occupancy Rate,Staffed All Beds - SOURCE,Staffed ICU Beds - SOURCE,Licensed All Beds - SOURCE,All Bed Occupancy Rate - SOURCE,ICU Bed Occupancy Rate - SOURCE,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID
0,IU HEALTH UNIVERSITY HOSPITAL,GENERAL ACUTE CARE,550 UNIVERSITY BLVD,NaN,INDIANAPOLIS,IN,46202,MARION,39.775283,-86.176562,...,NaN,None,None,NaN,None,None,100,NaN,NaN,100
1,HEALTHSOUTH NEW ENGLAND REHABILIATION AT BEVERLY,REHABILITATION,"800 CUMMINGS CENTER, 1ST FL, UNIT147",NaN,BEVERLY,MA,1915,ESSEX,42.560987,-70.887236,...,NaN,None,None,HIFLD-BEDS,None,None,10001915,NaN,NaN,10001915
2,ST. JOSEPH'S HOSPITAL,CRITICAL ACCESS,12886 TROXLER AVENUE,NaN,HIGHLAND,IL,62249,MADISON,38.754584,-89.667598,...,NaN,None,None,NaN,None,None,100062249,NaN,NaN,100062249
3,HOSP SAN CARLOS BORROMEO,GENERAL ACUTE CARE,550 CONCEPCION VERA AYALA,NaN,MOCA,PR,676,MOCA,18.390194,-67.109969,...,0.73,HCRIS-Total Staffed Beds,HCRIS-ICU Total Staffed Beds,DH-NUM_LICENS,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,1000676,4294.0,400111.0,1000676
4,RIVERVIEW MEDICAL CENTER,GENERAL ACUTE CARE,1 RIVERVIEW PLAZA,NaN,RED BANK,NJ,7701,MONMOUTH,40.353982,-74.063843,...,0.42,HCRIS-Total Staffed Beds,HCRIS-ICU Total Staffed Beds,DH-NUM_LICENS,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,10007701,2512.0,310034.0,10007701


In [7]:
nv_hosp = pd.read_csv(external_data_path('NV_facilities_can_20200428.csv'), encoding='utf-8')

In [8]:
nv_hosp.head()

,Hospital Name,Region,Tier,Zip Code,County,Hospital CMS Certification Number (CCN),Staffed all beds,Staffed ICU beds,Staffed all bed occupancy,Staffed ICU bed occupancy
0,Carson Tahoe Health,N,1,89703.0,Carson City,290019,159.0,24.0,54%,71%
1,Northern Nevada Medical Center,N,1,89434.0,Washoe,290032,124.0,16.0,52%,44%
2,Renown Regional,N,1,89521.0,Washoe,290001,610.0,92.0,67%,47%
3,Renown South Meadows,N,1,89521.0,Washoe,290049,76.0,8.0,41%,13%
4,Saint Mary's Regional Medical Center,N,1,89503.0,Washoe,290009,321.0,34.0,54%,53%


In [9]:
nv_hosp['Staffed all bed occupancy'] = nv_hosp['Staffed all bed occupancy'].apply(lambda x: float(str(x).replace('%',''))/100)
nv_hosp['Staffed ICU bed occupancy'] = nv_hosp['Staffed ICU bed occupancy'].apply(lambda x: float(str(x).replace('%',''))/100)

In [10]:
nv_hosp.head()

,Hospital Name,Region,Tier,Zip Code,County,Hospital CMS Certification Number (CCN),Staffed all beds,Staffed ICU beds,Staffed all bed occupancy,Staffed ICU bed occupancy
0,Carson Tahoe Health,N,1,89703.0,Carson City,290019,159.0,24.0,0.54,0.71
1,Northern Nevada Medical Center,N,1,89434.0,Washoe,290032,124.0,16.0,0.52,0.44
2,Renown Regional,N,1,89521.0,Washoe,290001,610.0,92.0,0.67,0.47
3,Renown South Meadows,N,1,89521.0,Washoe,290049,76.0,8.0,0.41,0.13
4,Saint Mary's Regional Medical Center,N,1,89503.0,Washoe,290009,321.0,34.0,0.54,0.53


In [11]:
ccm_matching_df.columns

Index(['Name', 'Hospital Type', 'Address', 'Address_2', 'City', 'State',
       'Zipcode', 'County', 'Latitude', 'Longitude', 'CCM_ID', 'DH-ID',
       'HCRIS-ID', 'HIFLD-ID', 'DH name lookup', 'DH address lookup',
       'HCRIS name lookup', 'HCRIS address lookup'],
      dtype='object')

In [12]:
ccm_nv = ccm_matching_df[ccm_matching_df['State'] == 'NV']
ccm_nv.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID,DH name lookup,DH address lookup,HCRIS name lookup,HCRIS address lookup
443,"DESERT PARKWAY BEHAVIORAL HEALTHCARE HOSPITAL,...",PSYCHIATRIC,3247 S MARYLAND PARKWAY,NaN,LAS VEGAS,NV,89109,CLARK,36.130593,-115.138230,111889109,6202.0,294013.0,111889109,Desert Parkway Behavioral Healthcare Hospital,"3247 S Maryland Pkwy, Las Vegas, NV",DESERT PARKWAY BEHAVIORAL HEALTHCARE,"3247 S MARYLAND PKWY, LAS VEGAS, NV"
446,HEALTHSOUTH REHABILITATION HOSPITAL OF LAS VEGAS,REHABILITATION,1250 S VALLEY VIEW BLVD,NaN,LAS VEGAS,NV,89102,CLARK,36.156732,-115.194223,111989102,4805.0,293026.0,111989102,Encompass Health Rehabilitation Hospital of La...,"1250 S Valley View Blvd, Las Vegas, NV",ENCOMPASS HEALTH REHABILITATION HOSP,"1250 SOUTH VALLEY VIEW BLVD., LAS VEGAS, NV"
451,KINDRED HOSPITAL AT ST ROSE DE LIMA CAMPUS,LONG TERM CARE,"102 E LAKE MEAD PARKWAY, THIRD FLOOR",NaN,HENDERSON,NV,89015,CLARK,36.038977,-114.984407,112189015,NaN,NaN,112189015,NaN,NaN,NaN,NaN
470,VA SOUTHERN NEVADA HEALTHCARE SYSTEM,MILITARY,1700 VEGAS DRIVE,NaN,LAS VEGAS,NV,89106,CLARK,36.189144,-115.162833,112789106,NaN,NaN,112789106,NaN,NaN,NaN,NaN
478,MOUNTAINVIEW HOSPITAL,GENERAL ACUTE CARE,3100 N TENAYA WAY,NaN,LAS VEGAS,NV,89128,CLARK,36.215852,-115.249107,113089128,2420.0,290039.0,113089128,MountainView Hospital,"3100 N Tenaya Way, Las Vegas, NV",MOUNTAIN VIEW HOSPITAL,"3100 N. TENAYA WAY, LAS VEGAS, NV"


In [13]:
search_ccm_nv = []
for idx, row in ccm_nv.iterrows():
    search_ccm_nv.append((', ').join(row[['Name', 
                                          'DH name lookup', 
                                          'HCRIS name lookup',
#                                           'County'
                                         ]].values.astype('str')).replace('nan',''))

In [14]:
search_ccm_nv_dict = {el:ccm_nv.iloc[idx]['CCM_ID'] for idx, el in enumerate(search_ccm_nv)}

In [15]:
nv_hosp.columns

Index(['Hospital Name', 'Region', 'Tier', 'Zip Code', 'County',
       'Hospital CMS Certification Number (CCN)', 'Staffed all beds',
       'Staffed ICU beds', 'Staffed all bed occupancy',
       'Staffed ICU bed occupancy'],
      dtype='object')

In [19]:
county = ''
for idx, row in nv_hosp.iterrows():
    search_str = (', ').join(nv_hosp.loc[idx,
                                        ['Hospital Name',
                                        ]].values.astype(str))
    
    result = process.extractOne(search_str, search_ccm_nv)

    if result != None:
        
        nv_hosp.loc[idx,'Matched Name'] = result[0]
        nv_hosp.loc[idx,'Matched Score'] = result[1]
        nv_hosp.loc[idx,'CCM_ID'] = search_ccm_nv_dict[result[0]]
        nv_hosp.loc[idx,'CCM County'] = ccm_nv[ccm_nv['CCM_ID'] == (search_ccm_nv_dict[result[0]])]['County'].values[0]


In [20]:
nv_hosp.shape

(61, 14)

In [21]:
nv_hosp.head()

,Hospital Name,Region,Tier,Zip Code,County,Hospital CMS Certification Number (CCN),Staffed all beds,Staffed ICU beds,Staffed all bed occupancy,Staffed ICU bed occupancy,Matched Name,Matched Score,CCM_ID,CCM County
0,Carson Tahoe Health,N,1,89703.0,Carson City,290019,159.0,24.0,0.54,0.71,"TAHOE PACIFIC HOSPITALS-NORTH, PAM Specialty H...",85.500008,113289434.0,WASHOE
1,Northern Nevada Medical Center,N,1,89434.0,Washoe,290032,124.0,16.0,0.52,0.44,"NORTHERN NEVADA MEDICAL CENTER, Northern Nevad...",90.000008,3389434.0,WASHOE
2,Renown Regional,N,1,89521.0,Washoe,290001,610.0,92.0,0.67,0.47,"RENOWN REGIONAL MEDICAL CENTER, Renown Regiona...",90.000008,5689502.0,WASHOE
3,Renown South Meadows,N,1,89521.0,Washoe,290049,76.0,8.0,0.41,0.13,"RENOWN SOUTH MEADOWS MEDICAL CENTER, Renown So...",90.000008,1389521.0,WASHOE
4,Saint Mary's Regional Medical Center,N,1,89503.0,Washoe,290009,321.0,34.0,0.54,0.53,"SAINT MARY'S REGIONAL MEDICAL CENTER, Saint Ma...",90.000008,1489503.0,WASHOE


In [22]:
nv_hosp[(nv_hosp['Matched Score'] > .90)][['Hospital Name','Matched Name']]

,Hospital Name,Matched Name
0,Carson Tahoe Health,"TAHOE PACIFIC HOSPITALS-NORTH, PAM Specialty H..."
1,Northern Nevada Medical Center,"NORTHERN NEVADA MEDICAL CENTER, Northern Nevad..."
2,Renown Regional,"RENOWN REGIONAL MEDICAL CENTER, Renown Regiona..."
3,Renown South Meadows,"RENOWN SOUTH MEADOWS MEDICAL CENTER, Renown So..."
4,Saint Mary's Regional Medical Center,"SAINT MARY'S REGIONAL MEDICAL CENTER, Saint Ma..."
...,...,...
56,Spring mountain sahara,"SPRING MOUNTAIN SAHARA, Spring Mountain Sahara..."
57,Spring Mountain Treatment Center,"SPRING MOUNTAIN TREATMENT CENTER, Spring Mount..."
58,DIgnity Health Rehab Hospital,HEALTHSOUTH REHABILITATION HOSPITAL OF LAS VEG...
59,Infinity Hospice Care,"CARSON TAHOE CONTINUING CARE HOSPITAL, Carson ..."


In [23]:
# save to file to do manual check and matching
nv_hosp.to_csv(processed_data_path('nv_facilitymatch_debug.csv'))

In [24]:
facility_matched_dict = {k:int(v) for k,v in nv_hosp[(nv_hosp['Matched Score'] >= 0)][['Hospital Name','CCM_ID']].values}

In [25]:
# manual matching override

# list of tuples as (ProviderName, CCM_ID)
manual_matches = [
    ("West Hills Behavioral Health (Failed to Report)", 189512),
    ("Carson Tahoe Continuing Care Hospital (LTACH)", 289703),
    ("Encompasshealth Rehab of Henderson", 689052),
    ("Kindred Lv - Sahara", 889119),
    ("Kindred Flamingo", 989146),
    ("st rose domincan hospital rose de lima campus", 1889015),
    ("ST ROSE SAN MARTIN", 1989113),
    ("St Rode Dominica - Siena", 2089052),
    ("Valley Hospital", 2489106),
    ("Carson Tahoe Health", 2689703),
    ("Seven Hills", 3689052),
    ("SNAMHS-Rawson Neal", 3889146),
    ("MGGH", 4989415),
    ("PAM Rehabilitation Hospital Centennial Hills", 176389149),
    ("VA Southern Nevada HCS", 5889086)
]
    
for p, ccm_id in manual_matches:
    facility_matched_dict[p] = ccm_id

facility_to_add = [
    ("Dignity Health - St. Rose Dominican Neighborhood Hospitals", "CCM-NV-0001"),
    ("Elite Medical Center", "CCM-NV-0002"),
    ("DIgnity Health Rehab Hospital", "CCM-NV-0003"),
    ("Infinity Hospice Care", "CCM-NV-0004"),
    ("Nathan Adelson Hospice", "CCM-NV-0005"),
]

for p, ccm_id in facility_to_add:
    facility_matched_dict[p] = ccm_id

In [26]:
manual_override_data = pd.read_csv(
    external_data_path('covidcaremap-ushcsc-facility-manual-override.csv'),
    dtype={'HCRIS-Provider Number': object, 
           'DH-OBJECTID': object,
           'CCM_ID': object}
)

In [27]:
manual_override_cols = list(manual_override_data.columns)
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'DH-OBJECTID',
 'HCRIS-Provider Number',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [28]:
ccm_csv_df.columns

Index(['Name', 'Hospital Type', 'Address', 'Address_2', 'City', 'State',
       'Zipcode', 'County', 'Latitude', 'Longitude', 'Staffed All Beds',
       'Staffed ICU Beds', 'Licensed All Beds', 'All Bed Occupancy Rate',
       'ICU Bed Occupancy Rate', 'Staffed All Beds - SOURCE',
       'Staffed ICU Beds - SOURCE', 'Licensed All Beds - SOURCE',
       'All Bed Occupancy Rate - SOURCE', 'ICU Bed Occupancy Rate - SOURCE',
       'CCM_ID', 'DH-ID', 'HCRIS-ID', 'HIFLD-ID'],
      dtype='object')

In [29]:
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'DH-OBJECTID',
 'HCRIS-Provider Number',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [30]:
col_replace = {
    'DH-OBJECTID': 'DH-ID',
    'HCRIS-Provider Number': 'HCRIS-ID',
    'HIFLD-ID': 'HIFLD-ID'
}

manual_override_cols = [col_replace.get(n, n) for n in manual_override_cols]

In [31]:
override_dict = {
    'Staffed All Beds - SOURCE': "CAN NHA 2020-04-28",
    'Staffed ICU Beds - SOURCE': "CAN NHA 2020-04-28",
    'All Bed Occupancy Rate - SOURCE': "CAN NHA 2020-04-28",
    'ICU Bed Occupancy Rate - SOURCE': "CAN NHA 2020-04-28",
    "Manual Override Reason": "More recent data from 2020-04-28",
    "Manual Override New Data Source": "CAN NHA"
}
list(override_dict.keys())

['Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [32]:
nv_hosp['Hospital Name']

0                      Carson Tahoe Health
1           Northern Nevada Medical Center
2                          Renown Regional
3                     Renown South Meadows
4     Saint Mary's Regional Medical Center
                      ...                 
56                  Spring mountain sahara
57        Spring Mountain Treatment Center
58           DIgnity Health Rehab Hospital
59                   Infinity Hospice Care
60                  Nathan Adelson Hospice
Name: Hospital Name, Length: 61, dtype: object

In [33]:
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'DH-ID',
 'HCRIS-ID',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [34]:
ccm_csv_df['CCM_ID']

0             100
1        10001915
2       100062249
3         1000676
4        10007701
          ...    
7141     99860622
7142      9993215
7143       999506
7144     99962703
7145      9998133
Name: CCM_ID, Length: 7146, dtype: object

In [36]:
l = []
not_added = []
for p in nv_hosp['Hospital Name']:
    
    ccm_id = str(facility_matched_dict[p])

    found_ccm_record = ccm_csv_df[ccm_csv_df['CCM_ID'] == ccm_id]
    
    # ignore adding new facilities for now, only update existing matched facilities
    if len(ccm_csv_df[ccm_csv_df['CCM_ID'] == ccm_id]) > 0:
#         hosp_occupancy = nv_hosp[nv_hosp['Hospital Name'] == p]['Bed Census'].values[0]/nv_hosp[nv_hosp['Hospital Name'] == p]['Total Staffed Bed Capacity'].values[0]
#         icu_occupancy = fl_icu[fl_icu['Hospital Name'] == p]['Adult ICU Census'].values[0]/fl_icu[fl_icu['Hospital Name'] == p]['Total AdultICU Capacity'].values[0]
        
        override_dict['Staffed All Beds'] = nv_hosp[nv_hosp['Hospital Name'] == p]['Staffed all beds'].values[0]
        override_dict['Staffed ICU Beds'] = nv_hosp[nv_hosp['Hospital Name'] == p]['Staffed ICU beds'].values[0]
        override_dict['All Bed Occupancy Rate'] = nv_hosp[nv_hosp['Hospital Name'] == p]['Staffed all bed occupancy'].values[0].astype(float)
        override_dict['ICU Bed Occupancy Rate'] = nv_hosp[nv_hosp['Hospital Name'] == p]['Staffed ICU bed occupancy'].values[0].astype(float)
    
        row_dict = {}

        for col in list(manual_override_cols):
#             print(col)
            # update data for col fields from FL
            if col in list(override_dict.keys()):
                row_dict[col] = override_dict[col]
            # add the other col fields unchanged
            elif len(found_ccm_record[col].values)>0:
                row_dict[col] = found_ccm_record[col].values[0]
            else:
                row_dict[col] = np.nan
        l.append(row_dict)
    else: not_added.append((p,ccm_id))

In [37]:
 not_added

[('Dignity Health - St. Rose Dominican Neighborhood Hospitals', 'CCM-NV-0001'),
 ('Elite Medical Center', 'CCM-NV-0002'),
 ('DIgnity Health Rehab Hospital', 'CCM-NV-0003'),
 ('Infinity Hospice Care', 'CCM-NV-0004'),
 ('Nathan Adelson Hospice', 'CCM-NV-0005')]

In [38]:
manual_override_data.shape

(3, 26)

In [39]:
new_overrides_df = pd.DataFrame(l)
manual_override_data = pd.concat([manual_override_data, new_overrides_df]).drop_duplicates('CCM_ID' , keep='last')

In [40]:
manual_override_data.iloc[-1]

Name                               SPRING MOUNTAIN TREATMENT CENTER
Hospital Type                                           PSYCHIATRIC
Address                              7000 WEST SPRING MOUNTAIN ROAD
Address_2                                                       NaN
City                                                      LAS VEGAS
State                                                            NV
Zipcode                                                       89117
County                                                        CLARK
Latitude                                                    36.1263
Longitude                                                  -115.246
Staffed All Beds                                                110
Staffed ICU Beds                                                  0
Licensed All Beds                                               110
All Bed Occupancy Rate                                         0.71
ICU Bed Occupancy Rate                          

In [41]:
manual_override_data['DH-OBJECTID'] = manual_override_data['DH-OBJECTID'].combine_first(manual_override_data['DH-ID'])
manual_override_data['HCRIS-Provider Number'] = manual_override_data['HCRIS-Provider Number'].combine_first(manual_override_data['HCRIS-ID'])

In [42]:
manual_override_data.drop(['DH-ID', 'HCRIS-ID'], inplace=True, axis=1)

In [43]:
manual_override_data.to_csv(external_data_path('covidcaremap-ushcsc-facility-manual-override.csv'), index=False)